In [1]:
%load_ext autoreload
%autoreload 2
from new_aeb_gplvm import *
from utils.data_generator import DataGenerator
from utils.myutils import Utils
from tqdm import trange
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import random
random.seed(42)

In [2]:
datagenerator = DataGenerator()
utils = Utils()

In [21]:
# dataset and model list / dict
dataset_list = [
    "01_ALOI",
    "02_annthyroid",
    "03_backdoor",
    "04_breastw",
    "05_campaign",
    "06_cardio",
    "07_Cardiotocography",
    "08_celeba",
    "30_satellite",
    "09_census",
    "99_linear",
    "99_circles",
    "99_moons",
    "99_clusters",
    "99_oilflow",
]
dataset = dataset_list[2]
datagenerator.dataset = dataset

data = datagenerator.generator(
    la=0.999,
    realistic_synthetic_mode=None,
    stdscale=False,
    minmax=True,
    noise_type=None,
)

Y_train = torch.tensor(data["X_train"], dtype=torch.float32)
Y_test = torch.tensor(data["X_test"], dtype=torch.float32)
lb_train = torch.tensor(data["y_train"], dtype=torch.float32)
lb_test = torch.tensor(data["y_test"], dtype=torch.float32)
idx_n = np.where(lb_train == 0)[0]
Y_train = Y_train[idx_n]
lb_train = lb_train[idx_n]

subsampling for dataset 03_backdoor...
current noise type: None
{'Samples': 10000, 'Features': 196, 'Anomalies': 236, 'Anomalies Ratio(%)': 2.36}


In [22]:
n_train = len(Y_train)
data_dim = Y_train.shape[1]
latent_dim = 10
n_inducing = 50
n_epochs = 1000
nn_layers = 15, 15
lr = 0.01
batch_size = 128

In [23]:
gplvm = AD_GPLVM(latent_dim, n_inducing, n_epochs, nn_layers, lr, batch_size)

In [24]:
gplvm.fit(Y_train)

Setando Kernel RBF Padrão


Loss: -230.61, iter no: 999: 100%|██████████| 1000/1000 [01:57<00:00,  8.50it/s]


In [27]:
score = gplvm.predict_score(Y_test)
score

array([[0.49420702],
       [0.02069123],
       [0.03975334],
       ...,
       [0.05084148],
       [0.24546959],
       [0.09754688]], dtype=float32)

In [28]:
utils.metric(y_true=lb_test, y_score=score)

{'aucroc': 0.6493876196750321, 'aucpr': 0.03336706483007417}